## Przygotowanie danych

In [2]:
import pandas as pd
import pickle
from dask_ml import preprocessing
import datetime

---------------------

In [31]:
!ls ../data

lablencoder.pickle			      partner_performances
partner_0A6A5C238CDDE180872512A932D6FAA8.csv  partners


In [3]:
df = pd.read_parquet('../data/partners/partner_235.parquet')

In [4]:
df

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,prod_category3,prod_category4,prod_category5,prod_category6,prod_category7,product_country,product_id,product_title,partner_id,user_id
14227,0,-1.00000,-1,2020-09-30,-1,0.00,0,3,401,0,...,0,0,0,0,0,10,1207576,0,235,6660437
15252,0,-1.00000,-1,2020-09-30,-1,0.00,0,3,0,0,...,0,0,0,0,0,10,663534,0,235,5440967
15440,0,-1.00000,-1,2020-09-30,-1,0.00,0,3,0,0,...,0,0,0,0,0,10,891718,0,235,2947222
15778,0,-1.00000,-1,2020-09-30,-1,0.00,0,3,0,0,...,0,0,0,0,0,10,577791,0,235,8803914
16046,0,-1.00000,-1,2020-09-30,-1,0.00,0,3,0,0,...,0,0,0,0,0,10,577791,0,235,10465337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974527,0,-1.00000,-1,2020-10-08,61,0.00,0,3,0,0,...,76,306,842,0,0,10,652435,769317,235,13353497
15974594,0,-1.00000,-1,2020-10-08,-1,0.00,0,3,0,0,...,0,0,0,0,0,10,792034,0,235,13276159
15974735,0,-1.00000,-1,2020-10-07,144,0.00,0,7,0,0,...,76,1304,179,172,0,10,1603700,16048,235,11483087
15974789,0,-1.00000,-1,2020-10-08,53,0.00,0,3,0,0,...,76,1553,72,0,0,10,515296,296887,235,9723165


In [60]:
grouped = df.groupby('click_timestamp')

In [61]:
days = {groupName:group for groupName, group in df.groupby('click_timestamp')}

In [71]:
def find_missing_dates(days_in_df_as_strings):
    from datetime import date, timedelta
    days_as_datetime = [datetime.datetime.strptime(x,"%Y-%m-%d") for x in days_in_df_as_strings]
    date_set = set(days_as_datetime[0] + timedelta(x) for x in range((days_as_datetime[-1] - days_as_datetime[0]).days))
    missing = sorted(date_set - set(days_as_datetime))
    return [str(x.date()) for x in sorted(list(date_set))], missing

In [72]:
day_keys_sorted, missing_days = find_missing_dates(list(days.keys()))

In [83]:
x = {'a':1, 'b':2}

In [84]:
x.get('a')

1

In [69]:
def fill_dict(missing, dictionary):
    df_headers = ['Sale','SalesAmountInEuro','time_delay_for_conversion','click_timestamp',
              'nb_clicks_1week','product_price','product_age_group','device_type',
              'audience_id','product_gender','product_brand',
              'prod_category1','prod_category2','prod_category3','prod_category4',
              'prod_category5','prod_category6','prod_category7','product_country',
              'product_id','product_title','partner_id','user_id']
    for x in missing:
        dictionary[str(x.date())] = pd.DataFrame(columns = df_headers)
    return dictionary

In [56]:
days.keys()

dict_keys(['2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-05', '2020-10-06', '2020-10-07', '2020-10-08', '2020-10-09', '2020-10-10', '2020-10-11', '2020-10-12', '2020-10-13', '2020-10-14', '2020-10-15', '2020-10-16', '2020-10-17', '2020-10-18', '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22', '2020-10-23', '2020-10-24', '2020-10-25', '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30', '2020-10-31', '2020-11-01', '2020-11-02'])

In [57]:
days = fill_dict(missing, days)

In [58]:
days.keys()

dict_keys(['2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-05', '2020-10-06', '2020-10-07', '2020-10-08', '2020-10-09', '2020-10-10', '2020-10-11', '2020-10-12', '2020-10-13', '2020-10-14', '2020-10-15', '2020-10-16', '2020-10-17', '2020-10-18', '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22', '2020-10-23', '2020-10-24', '2020-10-25', '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30', '2020-10-31', '2020-11-01', '2020-11-02', '2020-10-04'])

In [31]:
# u mnie grupy są stringiem i wyglądają tak, pobrałem je tak:
#         self.df_days_with_data = list(self.df_grouped_by_day.groups.keys())

days = ['2020-09-30', '2020-10-02', '2020-10-03', '2020-10-05', '2020-10-06', '2020-10-07', '2020-10-08', '2020-10-09', '2020-10-10', '2020-10-11', '2020-10-12', '2020-10-13', '2020-10-14', '2020-10-15', '2020-10-16', '2020-10-17', '2020-10-18', '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22', '2020-10-23', '2020-10-24', '2020-10-25', '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30', '2020-10-31', '2020-11-01', '2020-11-02']

-------------

In [3]:
from dask.distributed import Client, LocalCluster

In [4]:
cluster = LocalCluster()
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://127.0.0.1:40489 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.19 GB


In [2]:
df_headers = ['Sale','SalesAmountInEuro','time_delay_for_conversion','click_timestamp',
              'nb_clicks_1week','product_price','product_age_group','device_type',
              'audience_id','product_gender','product_brand',
              'prod_category1','prod_category2','prod_category3','prod_category4',
              'prod_category5','prod_category6','prod_category7','product_country',
              'product_id','product_title','partner_id','user_id']

dtypes={'prod_category1': 'category','prod_category2': 'category',
        'prod_category3': 'category','prod_category4': 'category',
        'prod_category5': 'category','prod_category6': 'category',
        'prod_category7': 'category', 'product_category': 'str'}

encoder_labels = ['product_age_group','device_type','audience_id',
                  'product_gender', 'product_brand','prod_category1','prod_category2',
                  'prod_category3','prod_category4','prod_category5',
                  'prod_category6','prod_category7','product_country',
                  'product_id','product_title','partner_id','user_id']

df = pd.read_csv("../../data/CriteoSearchData.csv", header=None,
                 names=df_headers, delimiter='\t', dtype=dtypes)

In [3]:
(len(df[df['Sale']>0])/len(df)) * 100

10.832462158111394

In [4]:
# potrzebne do labelencodera
df['product_title'] = df.product_title.astype(str)
df['product_title'] = df['product_title'].replace('nan', 'BROKEN')

df['product_title'] = df.product_title.astype(str)
df['user_id'] = df.user_id.astype(str)

In [4]:
df['click_timestamp'] = df['click_timestamp'].apply(lambda x: str(pd.to_datetime(int(x), unit='s').date()))

### Praca z dask_ml.preprocessing.LabelEncoder

#### Przygotowanie LabelEncodera

In [ ]:
# not needed anymore, load pickle file instead
labelEncoders = {x:preprocessing.LabelEncoder() for x in encoder_labels}
for k,v in labelEncoders.items():
    print('starting {}'.format(k))
    df[k] = v.fit_transform(df[k])
    print('done')

#### Zapis labelEncodera do pliku

In [31]:
import pickle
with open('../lablencoder_new.pickle', 'wb') as handle: #wiem, że tu jest literówka, trudno
    pickle.dump(labelEncoders, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Wykorzystanie labelEncodera

Załaduj przygotowany labelEncoder za pomocą:

In [2]:
!ls ../

dask-worker-space	   graph_plotter.ipynb	prepare_data	venv
data			   main.py		simulator
exclusion_logs_comparator  optimizer		Untitled.ipynb


In [1]:
import pickle
labelEncoders = pickle.load(open("../data/lablencoder.pickle","rb"))

In [2]:
labelEncoders.get('partner_id').transform("C306F0AD20C9B20C69271CC79B2E0887")

242

In [2]:
labelEncoders.get('partner_id').transform('04A66CE7327C6E21493DA6F3B9AACC7')

6

In [3]:
labelEncoders.get('partner_id').inverse_transform([235])

array(['C0F515F0A2D0A5D9F854008BA76EB537'], dtype=object)

In [4]:
labelEncoders

{'product_age_group': LabelEncoder(),
 'device_type': LabelEncoder(),
 'audience_id': LabelEncoder(),
 'product_gender': LabelEncoder(),
 'product_brand': LabelEncoder(),
 'prod_category1': LabelEncoder(),
 'prod_category2': LabelEncoder(),
 'prod_category3': LabelEncoder(),
 'prod_category4': LabelEncoder(),
 'prod_category5': LabelEncoder(),
 'prod_category6': LabelEncoder(),
 'prod_category7': LabelEncoder(),
 'product_country': LabelEncoder(),
 'product_id': LabelEncoder(),
 'product_title': LabelEncoder(),
 'partner_id': LabelEncoder(),
 'user_id': LabelEncoder()}

#### Badanie zawartości labelEncodera

In [11]:
labelEncoders

{'product_age_group': LabelEncoder(),
 'device_type': LabelEncoder(),
 'audience_id': LabelEncoder(),
 'product_gender': LabelEncoder(),
 'product_brand': LabelEncoder(),
 'prod_category1': LabelEncoder(),
 'prod_category2': LabelEncoder(),
 'prod_category3': LabelEncoder(),
 'prod_category4': LabelEncoder(),
 'prod_category5': LabelEncoder(),
 'prod_category6': LabelEncoder(),
 'prod_category7': LabelEncoder(),
 'product_country': LabelEncoder(),
 'product_id': LabelEncoder(),
 'product_title': LabelEncoder(),
 'partner_id': LabelEncoder(),
 'user_id': LabelEncoder()}

Translacja kolumn na wartości labelEncodera

In [7]:
for k,v in labelEncoders.items():
    df[k] = v.transform(df[k].values)
    print('{} done'.format(k))

product_age_group done
device_type done
audience_id done
product_gender done
product_brand done
prod_category1 done
prod_category2 done
prod_category3 done
prod_category4 done
prod_category5 done
prod_category6 done
prod_category7 done
product_country done
product_id done
product_title done
partner_id done
user_id done


In [ ]:
for k,v in labelEncoders.items():
    print("{} table lookup".format(k))
    print("showing {} classes:".format(len(v.classes_)))
    print(v.classes_)
    print("\n")

Kolumna product_title zawiera wartość BROKEN zastępującą wcześniej pojawiający sie NaN

In [37]:
'BROKEN' in labelEncoders.get('product_title').classes_

True

In [38]:
labelEncoders.get('product_title').transform(['BROKEN'])

array([596334])

In [39]:
labelEncoders.get('product_title').inverse_transform([596334])

array(['BROKEN'], dtype=object)

tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tornado/websocket.py", line 954, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/usr/local/lib/python3.8/dist-packages/tornado/web.py", line 3173, in wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/bokeh/server/views/ws.py", line 137, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


Eksport całego przeparsowanego datasetu:

In [10]:
df.to_parquet('CPStimestampFormattedCorrectly.parquet')

Plik per partner:

In [6]:
grouped = df.groupby('partner_id')

In [13]:
for v in df['partner_id'].unique():
    grouped.get_group(v).to_parquet('../../data/partners/partner_' + str(v) + '.parquet')